In [ ]:
import re
import glob
from itertools import zip_longest

import fitz
import hierarchy

In [ ]:
_reg = re.compile(r'(?!\d\s)([.])\s(?=[A-Z])')
def sentencize_dodf(s):
    sents = re.split(_reg, s)
    return [i[0] + (i[1] if not i[0].endswith('.') else '.') for i in
        zip_longest(sents[0::2], sents[1::2], fillvalue='.')]

def spaced_letters_fix(s):
    mts = re.split(r'((?:[A-ZÀ-Ž]{1,2}\s){3,})', s)
    offset = 0
    lis = [ s[:offset] ]
    for text, spaced in zip_longest(mts[0::2], mts[1::2], fillvalue=''):
        lis.append(text)
        lis.append(spaced.replace(' ', ''))
        lis.append(' ')
    lis.pop()    # last space is extra
    return ''.join(lis)



def drop_parenthesis(s):
    lis = re.split(r'([()])', s)
    ext_lis = lis + ['(']
    acc = 0
    new = []
    for tex, par in zip(ext_lis[0::2], ext_lis[1::2]):
        if tex:
            if not acc:
                new.append(tex)
        acc += (1 if par == '(' else -1)
    return ''.join(new)

def preprocess(s):
    s = drop_parenthesis(s)
    s = spaced_letters_fix(s)
    s = re.sub(r'\s{2,}', ' ', s)
    return s


In [ ]:
files = glob.glob('pdf/*.pdf')
# files

In [ ]:
doc_pdf = fitz.open(files[5])
h = hierarchy.mount_hierarchy(doc_pdf)
hierarchy.post_process_hierarchy(h);

In [ ]:
s = h['SEÇÃO I'][0][1][0][1]
# s

In [ ]:

for i in s[:-2]:
    print(i)
    print('\n')

In [ ]:
%%time
sentences = sentencize_dodf(i)
for sent in sentences:
    print(sent, end='\n\n\n')

In [ ]:
reg = r'[.:;]\s(?=[A-Z])'
l = [''.join(tup) for tup in
    zip_longest(re.split(reg, i), re.findall(reg, i), fillvalue='.')
]
for _ in l:
    print(_)
    print('----')

In [ ]:
dic=hierarchy.hierarchy_text(h)
del dic['SEÇÃO 0']
tt = []
for v in dic.values():
    tt.extend(v)
lis = [i for i in tt if not i == i.upper()]
no_break = [i.replace('\n', ' ') for i in lis]

In [ ]:
sentencize_dodf(preprocess( no_break[2] ))

In [ ]:
sentencize_dodf(preprocess( no_break[3] ))

In [ ]:
no_break